In [1]:
import re
import numpy as np
import pandas as pd
from string import punctuation
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/glluch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
allowed_punc = ['.', ',', '!', '@']
def remove_hash(text):
    return re.sub(r'#([^\s]+)', r'\1', text)

def remove_URL(text):
    return re.sub('((www\.[^r\s]+)|(https?://[^\rs]+))', 'URL', text)

def remove_punctuation(text):
    nopunc = [char for char in text if char in allowed_punc or char not in punctuation]
    return ''.join(nopunc)

def join_mention(text):
    return text.replace('@ ', '@')

def tokenize(text):
    return (word_tokenize(text))

def process(content): 
    text = content
    text = remove_hash(text)
    text = remove_URL(text)
    text = remove_punctuation(text)
    text = join_mention(text)
    
    return text

In [3]:
filepath = 'data/trumptweets.csv'

data = pd.read_csv(filepath)
data = data.sample(frac=0.3)
# print(data.head())
for i in range(5): 
    print(process(data.iloc[i]['content']))

@catisbetter666 @realDonaldTrump Im voting trump bc we need a real man to lead the country not that socialist Obama
 USAatUNGA  UNGApic.twitter.comMjIJ7HG3eU
 @TrumpFix @meganena13 @robertew2945 agreed Luisa. @realDonaldTrump WILL WIN! This is what he does!
Very interesting!URLshDSouzastatus1056315102327201793 …
 @DMBGreySt @realDonaldTrump @NYMag agreed. I wont be renewing. I got back issues that are paper weights. Bad people, false writing!


In [4]:
data['content_clean'] = data.apply(lambda row: process(row['content']), axis=1)
data.to_csv('data/trumptweets_clean.csv')

In [5]:
# from keras.preprocessing.text import Tokenizer

# num_words = 20000 
# tokenizer = Tokenizer(num_words=num_words, filters='')

# tokenizer.fit_on_texts(data['content_clean'])
# encoded = tokenizer.texts_to_sequences(data['content_clean'])
# flat_encoded = [enc for encoder in encoded for enc in encoder]
# total_words = len(tokenizer.word_index) + 1
# print(total_words)

In [6]:
from keras.preprocessing.text import Tokenizer

corpus = data['content_clean']
num_words = 5000
tokenizer = Tokenizer()
tokenizer = Tokenizer(num_words=num_words, filters='')

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_len = max([len(x) for x in input_sequences])
# max_sequence_len = 20
input_sequences = np.array(pad_sequences(input_sequences,   
                          maxlen=max_sequence_len, padding='pre'))

In [8]:
import keras.utils as ku 
import random 
random.shuffle(input_sequences)
predictors, labels = input_sequences[:,:-1],input_sequences[:,-1]
labels = ku.to_categorical(labels, num_classes=total_words)

In [9]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(150))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [10]:
import os
import tensorflow as tf
from keras.callbacks import EarlyStopping

early_stopper = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
# Checkpoing Model Weights
checkpoint_path = f'checkpoints/cp-?.ckpt'

checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1)


In [11]:
train_size = len(predictors) * 90 // 100
x, x_val = predictors[0:train_size], predictors[train_size:]
y, y_val = labels[0:train_size], labels[train_size:]

In [12]:
model = create_model(max_sequence_len, total_words)
model.fit(x, y, validation_data=(x_val, y_val), batch_size=512,
    epochs=10, verbose=2, callbacks=[early_stopper, checkpoint_cb])

Epoch 1/10

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt




Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001


Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt

Epoch 00001: saving model to checkpoints/cp-?.ckpt
323/323 - 129s - loss: 7.0506 - val_loss: 6.6949
Epoch 2/10

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epo


Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002


Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt

Epoch 00002: saving model to checkpoints/cp-?.ckpt
323/323 - 125s - loss: 6.6139 - val_loss: 6.5839
Epoch 3/10

Epoch 00003: saving model to checkpoints/cp-?.ckpt

Epoch 00003: saving model to checkpoints/cp-?.ckpt

Epoch 00003: saving model to checkpoints/cp-?.ckpt

Epo

...


Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008: saving model to checkpoints/cp-?.ckpt

Epoch 00008


Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009


Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009: saving model to checkpoints/cp-?.ckpt

Epoch 00009


Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010


Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010: saving model to checkpoints/cp-?.ckpt

Epoch 00010

In [13]:
import pickle

pickle.dump(tokenizer, open('model_states/tok.pkl', 'wb'))
model.save('model_states/test.h5')

In [14]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [21]:
text = generate_text('huge', 10, max_sequence_len, model)
print(text)

huge the great and and the great and and the great
